In [12]:
from utils import temporal_changes_utils, midas_db_utils, core_gene_utils, parse_patric, config
import pickle
from collections import defaultdict

In [3]:
snp_change_counts = pickle.load(open('../data_poyet/pickles/snp_change_dict.pkl', 'rb'))
gene_gain_counts = pickle.load(open('../data_poyet/pickles/gene_gain_dict.pkl', 'rb'))

species_spair_gene_gain_dict = defaultdict(dict)

for tup in snp_change_counts:
	for species in gene_gain_counts[tup]:
		count = snp_change_counts[tup][species]
		if count <= 20 and gene_gain_counts[tup][species] > 0:
			species_spair_gene_gain_dict[species][tup] = gene_gain_counts[tup][species]

In [ ]:
species_spair_gene_gain_full_dict = defaultdict(dict)

for species in species_spair_gene_gain_dict:
	print(species)
	temporal_change_map = temporal_changes_utils.load_temporal_change_map(species)
	for sample_i, sample_j in species_spair_gene_gain_dict[species]:
		gene_opps, gene_perr, gains, losses = temporal_changes_utils.calculate_gains_losses_from_temporal_change_map(temporal_change_map, sample_i, sample_j)
		if len(gains) != species_spair_gene_gain_dict[species][(sample_i, sample_j)]:
			print("Huh?")
		species_spair_gene_gain_full_dict[species][(sample_i, sample_j)] = gains

Bacteroides_ovatus_58035
Faecalibacterium_prausnitzii_62201
Bacteroides_fragilis_54507
Bacteroides_uniformis_57318
Bacteroides_vulgatus_57955


In [5]:
print(species_spair_gene_gain_full_dict)

defaultdict(<type 'dict'>, {'Bacteroides_ovatus_58035': {('SRR9224159', 'SRR9224160'): [('469590.5.peg.5671', 1.58, 42.17, 41.87, 40.87), ('665954.3.peg.679', 0.76, 42.17, 26.8, 40.87)], ('SRR9224293', 'SRR9224292'): [('469590.5.peg.5671', 1.68, 54.26, 34.66, 33.95)], ('SRR9224164', 'SRR9224165'): [('665954.3.peg.679', 2.17, 53.72, 39.74, 54.2)], ('SRR9224288', 'SRR9224253'): [('665954.3.peg.679', 0.39, 35.2, 12.9, 20.89)], ('SRR9224074', 'SRR9224020'): [('702443.3.peg.5025', 0.82, 53.11, 40.48, 55.1)], ('SRR9224022', 'SRR9224021'): [('702443.3.peg.3784', 0.7, 33.7, 33.83, 37.84)], ('SRR9224017', 'SRR9224405'): [('702443.3.peg.3784', 0.3, 46.15, 35.87, 39.82)], ('SRR9224025', 'SRR9224018'): [('702443.3.peg.21', 0.38, 36.57, 20.75, 33.21), ('702443.3.peg.2254', 1.36, 36.57, 22.44, 33.21), ('702443.3.peg.3784', 0.0, 36.57, 31.49, 33.21), ('702443.3.peg.5025', 1.77, 36.57, 23.53, 33.21), ('702443.3.peg.5106', 0.74, 36.57, 21.91, 33.21)], ('SRR9224021', 'SRR9224024'): [('702443.3.peg.5106'

In [8]:
species_spair_gene_gain_names_dict = defaultdict(dict)

for species in species_spair_gene_gain_full_dict:
    print(species)
    # get all genome ids for this species' pan genome:
    genome_ids = midas_db_utils.get_ref_genome_ids(species)
    
    # Load the non-shared genes (whitelisted genes):
    non_shared_genes = core_gene_utils.parse_non_shared_pangenome_genes(species)

    # load the gene descriptions for all genomes coresponding to this speceis:
    gene_descriptions=parse_patric.load_patric_gene_descriptions(genome_ids, non_shared_genes)
    
    for spair in species_spair_gene_gain_full_dict[species]:
        gene_names = []
        for gene_id, D1, Dm1, D2, Dm2 in species_spair_gene_gain_full_dict[species][spair]:
            try:
                gene_names.append(gene_descriptions[gene_id])
            except:
                print(gene_id)
        species_spair_gene_gain_names_dict[species][spair] = gene_names

Bacteroides_ovatus_58035
Faecalibacterium_prausnitzii_62201
Bacteroides_vulgatus_57955
Bacteroides_uniformis_57318
Bacteroides_fragilis_54507


In [19]:
f = open('%s/gene_gain_annotations.tsv' % config.analysis_directory, 'w')
f.write('\t'.join(['sample1', 'sample2', 'species', 'gain_count', 'gain_genes']) + '\n')
for species in species_spair_gene_gain_names_dict:
    for spair in species_spair_gene_gain_names_dict[species]:
        gain_count = species_spair_gene_gain_dict[species][spair]
        gain_genes = species_spair_gene_gain_names_dict[species][spair]
        f.write('\t'.join([spair[0], spair[1], species, str(gain_count), ','.join(gain_genes)]) + '\n')
f.close()

In [9]:
snp_change_counts = pickle.load(open('../data_poyet/pickles/snp_change_dict.pkl', 'rb'))
gene_loss_counts = pickle.load(open('../data_poyet/pickles/gene_loss_dict.pkl', 'rb'))

species_spair_gene_loss_dict = defaultdict(dict)

for tup in snp_change_counts:
	for species in gene_loss_counts[tup]:
		count = snp_change_counts[tup][species]
		if count <= 20 and gene_loss_counts[tup][species] > 0:
			species_spair_gene_loss_dict[species][tup] = gene_loss_counts[tup][species]

In [ ]:
species_spair_gene_loss_full_dict = defaultdict(dict)

for species in species_spair_gene_loss_dict:
	print(species)
	temporal_change_map = temporal_changes_utils.load_temporal_change_map(species)
	for sample_i, sample_j in species_spair_gene_loss_dict[species]:
		gene_opps, gene_perr, gains, losses = temporal_changes_utils.calculate_gains_losses_from_temporal_change_map(temporal_change_map, sample_i, sample_j)
		if len(losses) != species_spair_gene_loss_dict[species][(sample_i, sample_j)]:
			print("Huh?")
		species_spair_gene_loss_full_dict[species][(sample_i, sample_j)] = losses

Bacteroides_ovatus_58035
Bacteroides_fragilis_54507
Bacteroides_xylanisolvens_57185
Bacteroides_uniformis_57318
Bacteroides_vulgatus_57955


In [15]:
print(species_spair_gene_loss_full_dict)

defaultdict(<type 'dict'>, {'Bacteroides_ovatus_58035': {('SRR9224253', 'SRR9224258'): [('665954.3.peg.679', 12.9, 20.89, 2.35, 54.57)], ('SRR9224292', 'SRR9224291'): [('469590.5.peg.5671', 34.66, 33.95, 1.03, 24.5)], ('SRR9224160', 'SRR9224161'): [('469590.5.peg.5671', 41.87, 40.87, 1.36, 35.06), ('665954.3.peg.679', 26.8, 40.87, 0.0, 35.06)], ('SRR9224026', 'SRR9224025'): [('702443.3.peg.5025', 60.88, 64.21, 1.77, 36.57)], ('SRR9224024', 'SRR9224023'): [('702443.3.peg.5106', 25.88, 35.11, 0.76, 26.61)], ('SRR9224135', 'SRR9224462'): [('702443.3.peg.2254', 22.82, 37.14, 0.0, 32.98), ('702443.3.peg.5106', 24.45, 37.14, 0.6, 32.98)], ('SRR9224019', 'SRR9224022'): [('702443.3.peg.5025', 29.66, 27.16, 0.0, 33.7)], ('SRR9224172', 'SRR9224135'): [('702443.3.peg.3784', 29.3, 30.2, 0.0, 37.14)], ('SRR9224018', 'SRR9224017'): [('1339347.3.peg.168', 23.09, 33.21, 1.35, 46.15), ('702443.3.peg.21', 20.75, 33.21, 0.0, 46.15), ('702443.3.peg.3784', 31.49, 33.21, 0.3, 46.15)], ('SRR9224472', 'SRR922

In [16]:
species_spair_gene_loss_names_dict = defaultdict(dict)

for species in species_spair_gene_loss_full_dict:
    print(species)
    # get all genome ids for this species' pan genome:
    genome_ids = midas_db_utils.get_ref_genome_ids(species)
    
    # Load the non-shared genes (whitelisted genes):
    non_shared_genes = core_gene_utils.parse_non_shared_pangenome_genes(species)

    # load the gene descriptions for all genomes coresponding to this speceis:
    gene_descriptions=parse_patric.load_patric_gene_descriptions(genome_ids, non_shared_genes)
    
    for spair in species_spair_gene_loss_full_dict[species]:
        gene_names = []
        for gene_id, D1, Dm1, D2, Dm2 in species_spair_gene_loss_full_dict[species][spair]:
            try:
                gene_names.append(gene_descriptions[gene_id])
            except:
                print(gene_id)
        species_spair_gene_loss_names_dict[species][spair] = gene_names

Bacteroides_ovatus_58035
Bacteroides_xylanisolvens_57185
Bacteroides_vulgatus_57955
Bacteroides_uniformis_57318
Bacteroides_fragilis_54507


In [20]:
f = open('%s/gene_loss_annotations.tsv' % config.analysis_directory, 'w')
f.write('\t'.join(['sample1', 'sample2', 'species', 'loss_count', 'loss_genes']) + '\n')
for species in species_spair_gene_loss_names_dict:
    for spair in species_spair_gene_loss_names_dict[species]:
        loss_count = species_spair_gene_loss_dict[species][spair]
        loss_genes = species_spair_gene_loss_names_dict[species][spair]
        f.write('\t'.join([spair[0], spair[1], species, str(loss_count), ','.join(loss_genes)]) + '\n')
f.close()